<a href="https://colab.research.google.com/github/tsmfish/Notebooks/blob/main/rsa/rsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔐 Алгоритм шифрування RSA (Rivest–Shamir–Adleman)

**Мета:** Зрозуміти, як працює асиметричне шифрування, де для шифрування використовується один ключ (публічний), а для розшифрування — інший (приватний).

На відміну від Діффі-Хеллмана (який потрібен для створення *спільного* ключа), RSA дозволяє передавати зашифровані повідомлення безпосередньо.

### 📮 Аналогія: Поштова скринька

1.  **Публічний ключ (Замок):** Ви ставите на вулиці свою поштову скриньку і роздаєте всім відкриті навісні замки. Будь-хто може написати листа, покласти в скриньку і закрити її вашим замком.
2.  **Зашифроване повідомлення:** Після того як замок замкнено, ніхто (навіть той, хто відправив листа) не може його відкрити.
3.  **Приватний ключ (Ключ від замка):** Тільки у вас є унікальний ключ, який відкриває цей замок. Ви приходите і забираєте пошту.

![RSA](rsa-formula-split.png)

## 🧮 Математична основа RSA

Безпека RSA будується на складності **факторизації великих чисел**.
Легко перемножити два простих числа: $p \cdot q = n$.
Але дуже складно, знаючи величезне число $n$, знайти вихідні множники $p$ та $q$.

### Генерація ключів:

1.  Вибираємо два простих числа $p$ та $q$.
2.  Обчислюємо **модуль**:
    $$n = p \cdot q$$
    *(Число $n$ буде частиною і публічного, і приватного ключа).*
3.  Обчислюємо **функцію Ейлера** (кількість чисел, взаємно простих з $n$):
    $$\phi(n) = (p-1) \cdot (q-1)$$
4.  Вибираємо **відкриту експоненту** $e$. Вона має бути:
    * $1 < e < \phi(n)$
    * Взаємно простою з $\phi(n)$ (їхній НСД = 1).
    *(Зазвичай використовують 65537, але для прикладу візьмемо маленьке число).*
    **Публічний ключ:** $(e, n)$
5.  Обчислюємо **секретну експоненту** $d$. Це число, обернене до $e$ за модулем $\phi(n)$:
    $$d \cdot e \equiv 1 \pmod{\phi(n)}$$
    **Приватний ключ:** $(d, n)$

### Шифрування та Розшифрування:

Нехай $M$ — наше повідомлення (перетворене на число, $M < n$).

* **Шифрування (Публічним ключем):**
    $$C = M^e \pmod{n}$$
* **Розшифрування (Приватним ключем):**
    $$M = C^d \pmod{n}$$

## 🔢 Приклад "на серветці" (Ручний розрахунок)

Візьмемо дуже маленькі числа, щоб простежити математику.

1.  **Вибір простих чисел:**
    Нехай $p = 3$ та $q = 11$.

2.  **Модуль $n$:**
    $n = 3 \cdot 11 = 33$.

3.  **Функція Ейлера $\phi(n)$:**
    $\phi(33) = (3-1) \cdot (11-1) = 2 \cdot 10 = 20$.

4.  **Відкрита експонента $e$:**
    Потрібне число, взаємно просте з 20.
    Нехай $e = 3$ (оскільки НСД(3, 20) = 1).
    **Публічний ключ:** $\{3, 33\}$

5.  **Секретна експонента $d$:**
    Нам потрібно знайти $d$, таке що $(d \cdot 3) \pmod{20} = 1$.
    Підбором:
    * $3 \cdot 7 = 21$.
    * $21$ ділимо на $20$, остача $1$.
    Отже, $d = 7$.
    **Приватний ключ:** $\{7, 33\}$

---
**Тест:**
Шифруємо повідомлення (число) **M = 15**.

1.  **Шифрування:** $C = 15^3 \pmod{33}$
    $15^3 = 3375$
    $3375 = 33 \cdot 102 + 9$
    **Зашифроване повідомлення C = 9.**

2.  **Розшифрування:** $M = 9^7 \pmod{33}$
    $9^7 = 4782969$
    $4782969 \pmod{33} = 15$
    **Результат M = 15.**

Все спрацювало!

## 🧪 Експеримент: Чому не можна розшифрувати публічним ключем?

Давайте перевіримо, чи зможе зловмисник (або навіть сам відправник) прочитати повідомлення, якщо у нього є лише **публічний ключ**.

Ми спробуємо застосувати "функцію розшифрування", але замість секретного ключа $d$ підставимо публічний ключ $e$.

Математично ми спробуємо обчислити:
$$Attempt = C^e \pmod{n}$$

Очікуємо, що результат перетвориться на "сміття" і не збігатиметься з вихідним числом.

## 🧮 Математична перевірка "на серветці": Спроба злому

Давайте перевіримо це вручну, використовуючи наші числа з попереднього ручного прикладу.

**Нагадаємо дані:**
* Модуль $n = 33$
* Публічний ключ (експонента) $e = 3$
* Зашифроване повідомлення (Шифртекст) $C = 9$
* Оригінальне повідомлення було $M = 15$

**Гіпотеза хакера:**
"Якщо я зашифрував ключем $3$, то, можливо, я зможу ним же і розшифрувати?"

**Розрахунок:**
Спробуємо обчислити $Result = C^e \pmod{n}$.
$$Result = 9^3 \pmod{33}$$

1.  Підносимо до степеня:
    $$9^3 = 9 \cdot 9 \cdot 9 = 729$$

2.  Знаходимо остачу від ділення на 33:
    $$729 / 33 = 22.0909...$$
    Ціла частина — 22.
    $$22 \cdot 33 = 726$$
    $$729 - 726 = 3$$

**Результат:** $3$

**Порівняння:**
Ми отримали число **3**.
Наше вихідне повідомлення було **15**.

$$3 \neq 15$$

**Висновок:**
Використання публічного ключа для розшифрування перетворило дані на інше число, яке не має нічого спільного з оригіналом. "Замок" не може відкрити сам себе.

## 🐍 Реалізація на Python

У Python функція `pow(base, exp, mod)` вміє рахувати обернений елемент за модулем (знаходити $d$), якщо передати від'ємний степінь `-1`.

Напишемо простий клас `RSA_Toy` для демонстрації.

In [ ]:
import math

class RSA_Toy:
    def __init__(self, p: int, q: int) -> None:
        self.p: int = p
        self.q: int = q
        self.n: int = p * q
        self.phi: int = (p - 1) * (q - 1)
        self.e: int = self._choose_e()
        self.d: int = self._calculate_d()

    def _choose_e(self) -> int | None:
        # У реальному житті зазвичай e = 65537
        # Тут ми шукаємо маленьке e для наочності
        e = 3
        while e < self.phi:
            if math.gcd(e, self.phi) == 1:
                return e
            e += 2
        raise ValueError("Не вдалося знайти e")

    def _calculate_d(self) -> int:
        # Обчислюємо модульну мультиплікативну інверсію
        # d * e = 1 (mod phi)
        return pow(self.e, -1, self.phi)

    def get_public_key(self) -> int:
        return (self.e, self.n)

    def get_private_key(self) -> (int, int):
        return (self.d, self.n)

    def encrypt(self, message: int, public_key) -> int:
        e, n = public_key
        if message_int >= n:
            raise ValueError(f"Повідомлення занадто довге! Має бути < {n}")
        # Формула: C = M^e mod n
        encrypted = pow(message, e, n)
        return encrypted

    def decrypt(self, ciphertext: int) -> int:
        # Формула: M = C^d mod n
        decrypted = pow(ciphertext, self.d, self.n)
        return decrypted

# --- ДЕМОНСТРАЦІЯ ---

print("--- Генерація ключів ---")
# Використовуємо прості числа більші за 3 і 11, але все ще маленькі
rsa = RSA_Toy(p=61, q=53)

pub_key = rsa.get_public_key()
priv_key = rsa.get_private_key()

print(f"p = {rsa.p}, q = {rsa.q}")
print(f"Модуль n = {rsa.n}")
print(f"Функція Ейлера phi = {rsa.phi}")
print(f"Публічний ключ (e, n): {pub_key}")
print(f"Приватний ключ (d, n): {priv_key}  <-- НІКОМУ НЕ ПОКАЗУВАТИ!")

print("\n--- Шифрування ---")
secret_message = 1234  # Число має бути меншим за n (3233)
print(f"Вихідне повідомлення: {secret_message}")

# Шифруємо, використовуючи ПУБЛІЧНИЙ ключ
encrypted_msg = rsa.encrypt(secret_message, pub_key)
print(f"Зашифроване повідомлення (C): {encrypted_msg}")

print("\n--- Розшифрування ---")
# Розшифровуємо, використовуючи ПРИВАТНИЙ ключ (зберігається всередині об'єкта rsa)
decrypted_msg = rsa.decrypt(encrypted_msg)
print(f"Розшифроване повідомлення: {decrypted_msg}")

if secret_message == decrypted_msg:
    print("\n✅ Успіх! Повідомлення відновлено.")
else:
    print("\n❌ Помилка! Щось пішло не так.")

In [ ]:
print("--- Спроба злому публічним ключем ---")
encrypted_msg = 1234
# Беремо зашифроване повідомлення з попереднього кроку
print(f"Маємо шифртекст: {encrypted_msg}")

# Отримуємо параметри публічного ключа
e_public, n_public = {1, 2}

# Спробуємо 'розшифрувати', використовуючи e (частину публічного ключа) замість d
# Формула невдалої спроби: Attempt = (C^e) % n
failed_attempt = pow(encrypted_msg, e_public, n_public)

print(f"Результат спроби розшифрування публічним ключем: {failed_attempt}")
print(f"Оригінальне повідомлення: {secret_message}")

# Перевірка
if failed_attempt == secret_message:
    print("\n😱 О ні! Повідомлення розшифровано (це математично неможливо для коректного RSA).")
else:
    print("\n🔒 Безпечно! Результат не збігається з оригіналом.")
    print("Це доводить, що знаючи ключ шифрування, неможливо розшифрувати повідомлення.")

## ⚠️ Важливе про безпеку ("Навчальний" vs "Реальний" RSA)

Код вище — це так званий **"Textbook RSA"** (Навчальний RSA). Його **не можна** використовувати для захисту реальних даних. Чому?

1.  **Детермінованість:** У навчальному RSA одне й те саме повідомлення завжди шифрується в один і той самий шифртекст. Якщо зловмисник здогадається, що ви відправляєте слово "Привіт", він може зашифрувати "Привіт" вашим публічним ключем і порівняти результат з перехопленим повідомленням.
2.  **Малі числа:** Ми використовували 2-3 значні прості числа. Комп'ютер зламає такий ключ за мікросекунди. Реальний RSA використовує ключі довжиною 2048 або 4096 біт (це числа з 600+ знаками).
3.  **Немає Padding (Доповнення):** У реальних системах (наприклад, у бібліотеці `cryptography` для Python) використовується схема **OAEP**. Вона додає до повідомлення випадкове "сміття" перед шифруванням, щоб кожного разу шифртекст виглядав по-новому, навіть для однакових повідомлень.